## Huntbook
# Tracing Empire Remote Code Execution to Add Backdoor User (via WMI)

This interactive Kestrel huntbook is about tracing an adversary, remoteley executing code to add a backdoor user on the target system with the WMI.

## Dataset Description

The dataset is showing, how the WMI is used to execute code remotely. The focus is on the use of the WMI Win32_Process class and the method "Create".

Dataset: https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_wmic_add_user_backdoor.zip

## Adversary View
```
(Empire: agents) > 
[*] Sending POWERSHELL stager (stage 1) to 172.18.39.5
[*] New agent 6Z78CY25 checked in
[+] Initial agent 6Z78CY25 from 172.18.39.5 now active (Slack)
[*] Sending agent (stage 2) to 6Z78CY25 at 172.18.39.5
agents

[*] Active agents:

Name     La Internal IP     Machine Name      Username                Process            PID    Delay    Last Seen            Listener
----     -- -----------     ------------      --------                -------            ---    -----    ---------            ----------------
KFL6CMNZ ps 172.18.39.5     WORKSTATION5      *THESHIRE\pgustavo      powershell         7584   5/0.0    2020-09-14 11:33:59  http            
YGBLW8EM ps 172.18.39.5     WORKSTATION5      *THESHIRE\wardog        powershell         8924   5/0.0    2020-09-14 11:40:53  http            
UBCKLYFA ps 172.18.39.5     WORKSTATION5      *THESHIRE\pgustavo      powershell         5412   5/0.0    2020-09-14 11:57:16  http            

6Z78CY25 ps 172.18.39.5     WORKSTATION5      *THESHIRE\pgustavo      powershell         9564   5/0.0    2020-09-14 12:02:08  http            

(Empire: agents) > interact 6Z78CY25
(Empire: 6Z78CY25) > 
(Empire: 6Z78CY25) > shell wmic /node:WORKSTATION6 process call create "net user /add backdoor pa$$w0rd1"
[*] Tasked 6Z78CY25 to run TASK_SHELL
[*] Agent 6Z78CY25 tasked with task ID 1
(Empire: 6Z78CY25) > 
Executing (Win32_Process)->Create()

Method execution successful.

Out Parameters:
instance of __PARAMETERS
{
  ProcessId = 7768;
  ReturnValue = 0;
};

..Command execution completed.

(Empire: 6Z78CY25) >

```

Source: https://securitydatasets.com/notebooks/atomic/windows/execution/SDWIN-200914080546.html

## Tactics & Techniques

For the attack, the tactics [TA0002](https://attack.mitre.org/tactics/TA0002/),[TA0008](https://attack.mitre.org/tactics/TA0008/) and techniques [T1047](https://attack.mitre.org/techniques/T1047/) are used.

The adversary uses Lateral Movement to enter and control the system remotely and Execution techniques to run code. The Windows Management Instrumentation (WMI) is abused to execute the malicious commands. 


## Get Processes and Filter for "WMIC"

First we look for all processes from the dataset, that are connected to activities with WMIC.

WMIC is a tool that allows command-line access to the WMI. With this tool, it is possible to query the operating system for detailed information about installed hardware and Windows settings, run management tasks, and even execute other programs or commands. In this case it was used for the last case, executing commands.

In [19]:
procs = GET process from https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_wmic_add_user_backdoor.zip 
where command_line LIKE '%WMIC%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
procs,process,1,154,170,285,293,4,2,174,5,31,170,170


We found records to one process entity. To examine the process further, we display it.

In [20]:
DISP procs ATTR pid, x_unique_id, created, name, command_line 

sproc = procs

pid 
 x_unique_id 
 created 
 name 
 command_line 
 
 
 
 
 9428 
 {2d351099-5ca9-5f5f-2f04-000000000400} 
 2020-09-14T12:06:01.826Z 
 WMIC.exe 
 "C:\windows\System32\Wbem\WMIC.exe" /node:WORKSTATION6 process call create "net user /add backdoor paw0rd1" 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 sproc 
 process 
 1 
 154 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

In the process we see, thet WMIC.exe is used, to create a process on WORKSTATION6 to execute the malicious code.

## Analyze the Process
In the next hunting step, we analyze the suspicious process we found.

To find related processes, we need to look for the WMI Provider Host (`WmiPrvSE.exe`).

In [21]:
wmiprovhost = GET process WHERE command_line like '%wmiprvse%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
wmiprovhost,process,2,258,280,468,513,0,0,292,14,30,280,280


In [22]:
DISP wmiprovhost ATTR pid, x_unique_id, created, name, command_line 

pid,x_unique_id,created,name,command_line
3952,{83d0c8c3-5caa-5f5f-f002-000000000400},2020-09-14T12:06:02.209Z,WmiPrvSE.exe,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding
7296,{2d351099-5cb0-5f5f-3804-000000000400},2020-09-14T12:06:08.432Z,WmiPrvSE.exe,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding


Looking at the time stamp, we see, that the processes with `pid` "3952" and "7296" are created some seconds later.

We can now drill it down even further to check, which processes are created by the WIM Provider Host.

In [23]:
processes = FIND process CREATED BY wmiprovhost

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
processes,process,1,45,301,490,554,0,0,317,17,30,301,301


In [24]:
DISP processes ATTR pid, x_unique_id, created, name, command_line, parent_ref.pid, parent_ref.name

pid,x_unique_id,created,name,command_line,parent_ref.pid,parent_ref.name
7768,{83d0c8c3-5caa-5f5f-f102-000000000400},2020-09-14T12:06:02.316Z,net.exe,net user /add backdoor paw0rd1,3952,WmiPrvSE.exe


We now found the process with the malicious code execution that adds the backdoor user. With the parent_ref information we see, that it is created by the WIM Provider Host we checked before.